In [1]:
import random
import itertools
import numpy as np
import pandas as pd
from collections import Counter
from generate_pick_rules import generate_pick_rules

## 1. データの準備

match_picks.csvに2019年で行われた各試合のチャンピオンピックをまとめている

In [2]:
# You can get a "match_picks.csv" by running the "get_picks_info.py"
df = pd.read_csv("match_picks.csv")
df.head()

,date,league,top,jungle,mid,bottom,support
0,2019-03-24,LCS,Lissandra,Nocturne,Irelia,Ezreal,Rakan
1,2019-03-24,LCS,Poppy,Jarvan IV,Ryze,Lucian,Braum
2,2019-03-24,LCS,Sylas,Sejuani,Irelia,Twitch,Galio
3,2019-03-24,LCS,Jax,Nunu & Willump,Ryze,Vayne,Braum
4,2019-03-24,LCS,Vladimir,Jarvan IV,Orianna,Varus,Tahm Kench


#### チャンピオンとポジションごとのピック回数をカウントしてみる

In [3]:
def count_picks(df):
    result = pd.DataFrame(columns=["champions", "picks_count", "positions"])

    positions = ["top", "jungle", "mid", "bottom", "support"]
    for position in positions:
        temp = pd.DataFrame(
            Counter(df[position]).items(),
            columns=["champions", "picks_count"]
        )
        temp["positions"] = position
        result = pd.concat([result, temp]).reset_index(drop=True)

    return result

In [4]:
df_count = count_picks(df)
df_count.head()

,champions,picks_count,positions
0,Lissandra,8,top
1,Poppy,66,top
2,Sylas,63,top
3,Jax,48,top
4,Vladimir,197,top


## 2. バスケット分析

- ピックの組み合わせは、買い物かごに入れた商品の組み合わせだと考えられる
- pyfpgrowthライブラリーを使って、ピックの結果から有用なルールを抽出

In [5]:
# You can modify the code of a "generate_pick_rules.py" if you want.
df_rules = generate_pick_rules(df)
df_rules.head()

,picked,recommend,confidence,lift
0,"(Maokai,)",Nocturne,1.0,32.117647
1,"(Elise, Rengar)",Twisted Fate,1.0,84.000000
2,"(Elise, Twisted Fate)",Rengar,1.0,819.000000
3,"(Rengar, Twisted Fate)",Elise,1.0,20.475000
4,"(Talon,)",Sylas,0.5,4.321900


## 3. ピックを生成するための関数を作成

#### まずはある程度ランダムで最初のピックを生成

In [6]:
def generate_first_pick(df):
    positions = ["top", "jungle", "mid", "bottom", "support"]
    first_pick_position = random.choice(positions)

    df = df.loc[df["positions"] == first_pick_position, :]
    df = df.sort_values(by="picks_count", ascending=False).reset_index(drop=True)
    n = np.random.randint(0, 2)

    return {first_pick_position: df.iloc[n, 0]}

例えば、こんな形で抽出される

In [9]:
for i in range(5):
    print(generate_first_pick(df_count))

{'mid': 'Corki'}
{'jungle': 'Sejuani'}
{'top': 'Kennen'}
{'bottom': "Kai'Sa"}
{'bottom': 'Ezreal'}


#### 前回までのピック情報をもとに次のピックを生成

In [10]:
def add_next_pick(df_count, df_rules, picks):
    for i, _ in enumerate(list(picks.values()), 1):
        for picked in itertools.permutations(list(picks.values()), len(picks) - i + 1):
            rules = df_rules.loc[df_rules["picked"] == str(picked), :] \
                            .sort_values(by="lift", ascending=False) \
                            .reset_index(drop=True)
            if len(rules) > 0:
                for i, next_pick in enumerate(rules["recommend"]):
                    if next_pick in picks.values():
                        continue
                    possible_positions = df_count[
                        df_count["champions"] == next_pick
                    ].sort_values(by="picks_count", ascending=False)[
                        "positions"
                    ].values
                    for position in possible_positions:
                        if position not in picks.keys():
                            picks[position] = next_pick
                            return picks

    return add_random_pick(df_count, picks)

毎回同じ結果になることを防ぐために、ある程度ランダム要素を入れておく

In [11]:
def add_random_pick(df, picks):
    positions = ["top", "jungle", "mid", "bottom", "support"]
    positions = list(set(positions) - set(picks.keys()))
    champions = df[df["positions"].isin(positions)]
    champions = champions[champions["picks_count"] > 10].values

    while True:
        next_pick = random.choice(champions)
        next_pick_champion = next_pick[0]
        next_pick_position = next_pick[2]
        if next_pick_champion not in picks.values():
            break
    picks[next_pick_position] = next_pick_champion

    return picks

例えば、こんな形で抽出される

In [13]:
for i in range(3):
    print(add_next_pick(df_count, df_rules, {'bottom': 'Ezreal'}))

{'bottom': 'Ezreal', 'support': 'Taric'}
{'bottom': 'Ezreal', 'jungle': "Kha'Zix"}
{'bottom': 'Ezreal', 'jungle': 'Aatrox'}


## 4. 仮想ピックを生成

今まで作成した関数を組み合わせて仮想ピックを生成

In [14]:
def generate_virtual_picks(df_count, df_rules, n=5):
    df = pd.DataFrame(
        columns=["top", "jungle", "mid", "bottom", "support"]
    )
    for i in range(n):
        picks = generate_first_pick(df_count)
        while (len(picks) < 5):
            picks = add_next_pick(df_count, df_rules, picks)
        temp = pd.DataFrame([picks])
        temp = temp[["top", "jungle", "mid", "bottom", "support"]]
        df = pd.concat([df, temp]).reset_index(drop=True)
    return df

#### 実際にありそうなピックを生成できた

In [15]:
result = generate_virtual_picks(df_count, df_rules, 10)
result

,top,jungle,mid,bottom,support
0,Irelia,Qiyana,Ryze,Ezreal,Alistar
1,Gangplank,Jarvan IV,Jayce,Xayah,Gragas
2,Gangplank,Poppy,Corki,Xayah,Tahm Kench
3,Urgot,Taliyah,Aatrox,Sivir,Braum
4,Ryze,Zac,Zed,Kai'Sa,Rakan
5,Aatrox,Xin Zhao,Kayle,Lucian,Morgana
6,Neeko,Nidalee,Lissandra,Jinx,Lux
7,Jax,Rek'Sai,Lissandra,Viktor,Tahm Kench
8,Kennen,Sejuani,Qiyana,Vayne,Nautilus
9,Kayle,Nidalee,Rumble,Sona,Tahm Kench
